In [ ]:
# --coding:utf-8
# @author:Liu Ziyu
# Date:2020-04-22

import os
import glob
import numpy as np
import math
import matplotlib.image as mpimg 
import matplotlib.pyplot as plt
import time
import csv

SConst = 1367
beta = 0.5
Div_skyazi = 16
day = [31,28,31,30,31,30,31,31,30,31,30,31]
# optical air mass from Smithsonian meteorological tables v.114
m_thita = [5.6,6.18,6.88,7.77,8.9,10.39,12.44,15.36,19.79,26.96]

# get all pano files under a rootpath
def get_allpano(cwd):
    result = []
    get_dir = os.listdir(cwd)  
    for i in get_dir:          
        sub_dir = os.path.join(cwd,i)  
        if os.path.isdir(sub_dir):     
            if sub_dir.endswith("_pano"):
                result.append(i)
    return result


# get all BSV files under a rootpath
def get_allBSV(cwd):
    result = []
    get_dir = os.listdir(cwd)  
    for i in get_dir:          
        sub_dir = os.path.join(cwd,i)  
        if os.path.isdir(sub_dir):     
            if sub_dir.endswith("BSV"):
                result.append(i)
    return result

# get all img under a rootpath
def get_allimg(cwd):
    paths = glob.glob(os.path.join(cwd, '*.png'))
    paths.sort()
    return paths


# extract the roadid, lat, lon from the imgname and return as array
def get_write_imginfo(imgname):
    imgname = imgname.rsplit('/',1)[1]
    temp = imgname.split('_')
    roadid = temp[0]
    lat = temp[1]
    lng = temp[2].rsplit('.',1)[0]
    return [roadid,lat,lng]


#split sunmaparr for center points in a year by month
def split_sunmaparr_c(sunmaparr_cpath):
    sunmaparr_c = np.loadtxt(sunmaparr_cpath,dtype=np.float,delimiter=',',encoding='utf-8')
    # split the sunmap by months
    intervalarrm = np.arange(2,13,1)
    split_atmc = sunmaparr_c[:, 0].searchsorted(intervalarrm)
    sunmaparr_c = np.split(sunmaparr_c, split_atmc)
    return sunmaparr_c

def split_skymaparr(skymap_path):
    skymaparr = np.loadtxt(skymap_path,dtype=np.float,delimiter=',',encoding='utf-8',skiprows=1)
    # split the sunmap by months
    intervalarrm = np.arange(1,128,1)
    split_at = skymaparr[:, 1].searchsorted(intervalarrm)
    skymaparr = np.split(skymaparr, split_at)
    return skymaparr

#split sunmaparr for each hour interval in a year by month and day hour   
def split_sunmaparr_y(sunmaparr_ypath):
    sunmaparr_y = np.loadtxt(sunmaparr_ypath,dtype=np.float,delimiter=',',encoding='utf-8',skiprows=1)
    intervalarrm = np.arange(2,13,1)
    split_atmy = sunmaparr_y[:, 0].searchsorted(intervalarrm)
    sunmaparr_y = np.array(np.split(sunmaparr_y, split_atmy))
    # split the sunmap by hour interval
    for i in range(0,len(sunmaparr_y)):
        # get the sunrise and sunset time for each day
        maxh = sunmaparr_y[i][:,3].max()
        minh = sunmaparr_y[i][:,3].min()
        # print(minh,maxh)
        if math.modf(maxh)[0] > 0.5:
            maxh = round(maxh)
        elif math.modf(maxh)[0] < 0.5:
            maxh = math.ceil(maxh)-0.5
        if math.modf(minh)[0] > 0.5:
            minh = round(minh)
        elif math.modf(minh)[0] < 0.5:
            minh = round(minh) + 0.5
        else:
            minh = minh + 0.5
        # sort the array by hour inter
        intervalarr = np.arange(minh, maxh, 0.5, dtype = float)
        temparr = np.array(sorted(sunmaparr_y[i],key=lambda x:(x[3])))
        split_ath = temparr[:, 3].searchsorted(intervalarr)
        sunmaparr_y[i] = np.split(temparr, split_ath)
#         print(sunmaparr_y[i])
    return sunmaparr_y


def get_gap(img,sunmapy_arr,hour_interval):
    blue_rgb = np.array([6, 230, 230],dtype = np.float)
    gap_sunmap = []
    hour_sundur = np.empty((len(sunmapy_arr)), dtype = np.float)
    for j in range(0,len(sunmapy_arr)):
#         print(len(sunmapy_arr))
        sundur = 0.0
        hour_gaparr = np.empty((len(sunmapy_arr[j])), dtype = np.float)
        for k in range(0,len(sunmapy_arr[j])):
#             print(len(sunmapy_arr[j]))
            count = 0
            ptnum = sunmapy_arr[j][k].shape[0]
            for pt in range(0,ptnum):
                x = int(sunmapy_arr[j][k][pt][9])
                y = int(sunmapy_arr[j][k][pt][10])
                if((image[x-1][y-1]*255 == blue_rgb).all()):
                    count += 1
            #print(count,ptnum)
            hour_gaparr[k] = count / ptnum * 1.0
            sundur += hour_gaparr[k] * hour_interval * len(sunmapy_arr[j])
        gap_sunmap.append(hour_gaparr)
        hour_sundur[j] = sundur
    return gap_sunmap, hour_sundur


def get_skygap(img,skymap_arr):
    blue_rgb = np.array([6, 230, 230],dtype = np.float)
    gap_skymap = []
#     hour_sundur = np.empty((len(sunmapy_arr)), dtype = np.float)
    for j in range(0,len(skymap_arr)):
        count = 0
        ptnum = skymap_arr[j].shape[0]
        for pt in range(0,ptnum): 
            x = int(skymap_arr[j][pt][4])
            y = int(skymap_arr[j][pt][5])
            if((image[x-1][y-1]*255 == blue_rgb).all()):
                count += 1
            #print(count,ptnum)
        gap_skymap.append(count / ptnum * 1.0)
    return gap_skymap


def deg2rad_G(data):
    if data == '':
        data = 0
    else:
        data = math.radians(float(data))
    return data


if __name__ == "__main__": 
    city = ['AL','FCG','JL','CD','JQ','NJZ','LS','QQHE','XA','ZJ','ZT','HEB','HZ','SZ','TM','MDJ']
    cityi = 15
    # AL_30.3_81.1sunmapcenter.csv fcg_21.7_108.4sunmapcenter.csv JL_43.9_126.6sunmapcenter.csv CD_30.6_104.1sunmap.csv
    # JQ_39.7_98.5sunmap.csv NJZ_25.8_98.9sunmap.csv LS_29.7_91.1sunmap.csv QQHE_47.4_123.9sunmap XA_34.3_108.9sunmap.csv
    # ZJ_21.3_110.4sunmap ZT_27.3_103.7sunmap.csv HEB_45.8_126.5sunmap.csv HZ_24.4_111.6sunmap.csv SZ_22.5_114.1sunmap.csv
    # TM_30.7_113.2sunmap.csv MDJ_44.6_129.6sunmap.csv
    rootpath = '/datadrive/lzy/Pano/'
    trafficpath = '/datadrive/lzy/Traffic/'+city[cityi]+'/'+city[cityi]+'.csv'
    cloudpath = '/datadrive/lzy/Traffic/'+city[cityi]+'/'+city[cityi]+'cloudy.csv'
    skymappath = '/datadrive/lzy/skymap_az.csv'
    outpath = '/datadrive/lzy/R/'
    data = np.loadtxt(trafficpath,dtype=np.str,delimiter=',',encoding='utf-8')
    cloudy =  np.loadtxt(cloudpath,dtype=np.str,delimiter=',',encoding='utf-8')
    # read the sunmap for centerpoint and all for given time interval like 0.05
    sunmapc_path = '/datadrive/lzy/Traffic/'+city[cityi]+'/MDJ_44.6_129.6sunmapcenter.csv'
    sunmapy_path = '/datadrive/lzy/Traffic/'+city[cityi]+'/MDJ_44.6_129.6sunmap.csv'
    sunmapc_arr = split_sunmaparr_c(sunmapc_path)
    sunmapy_arr = split_sunmaparr_y(sunmapy_path)
    skymap_arr = split_skymaparr(skymappath)
    sky_num = len(skymap_arr)
    
    # skymap: weight the proportion of diffuse radiation originating in a given sky sector relative to all sectors 
    skyweight_arr = []
    # the thita and azimuth of center point for each sector
    skythitac_arr = []
    skyazimuthc_arr = []
    for nsector in range(0,sky_num):
        thita_max = np.max(skymap_arr[nsector][:,15])
        thita_min = np.min(skymap_arr[nsector][:,15])
        thita_avg = np.mean(skymap_arr[nsector][:,15])
        azimuth_avg = np.mean(skymap_arr[nsector][:,16])
        # Weightθ,α = (cosθ2- cosθ1) / Divazi    (7)
        skyweight = (math.cos(math.radians(thita_min))-math.cos(math.radians(thita_max)))/Div_skyazi
        skyweight_arr.append(skyweight)
        skythitac_arr.append(math.radians(thita_avg))
        skyazimuthc_arr.append(math.radians(azimuth_avg))          
    
#     BSVfile = get_allBSV(rootpath)
    BSVfile = ['MDJBSV']
    for rootf in BSVfile:
        panopath = rootpath + rootf
        outname = outpath + rootf + '_r.csv'
        outfile = open(outname, 'w' , newline="")
        writer = csv.writer(outfile, lineterminator='\n')
        writer.writerow(['fid','roadid','roadtype','lat','lng','edm','slope','aspect','dir_nt','dir_work',
                         'dir_wkend','dir_weighted','dif_nt','dif_work','dif_wkend','dif_weighted','total_nt',
                         'total_work','total_wkend','total_weighted'])
        panofile = get_allpano(panopath)
        for panof in panofile:
            imgpath = panopath + '/' + panof
            print(imgpath)
            imgname = get_allimg(imgpath)
#             imgname = get_allimg("/datadrive/urbanplayground/Streetview/SegDone/ALBSV/SegPic/ALBSV5_p_pano")
            # sparse image name
            for img in imgname:
                imginfo = get_write_imginfo(img)
                f_id = int(imginfo[0])
                image = mpimg.imread(img)
                #plt.imshow(image)
                #plt.show()
                result = get_gap(image,sunmapy_arr,0.5)
                gap_sunmap = result[0]
#                 print(gap_sunmap)
                sundur_img = result[1]
                gap_sky = get_skygap(image,skymap_arr)
                # get the elevation of this image
                Elev = data[f_id-1,5]
                if Elev == '' or Elev == -1:
                    Elev = float(data[f_id-2,5])
                else:
                    Elev = float(Elev)
                # print(data[f_id-1,5])
                # Elev = float(data[f_id-1,5])
                # slope Gz aspect Ga
                Gz = deg2rad_G(data[f_id-1,6])
                Ga = deg2rad_G(data[f_id-1,7])
                # m(θ) is the relative optical path length
                mz_y = []
                AngIn_y = []
                # every sector Direct radiation workday
                Dir_ywork = []
                # each month of the total direct radiation workday
                Dir_monwork = []
                # the total direct of the year workday
                Dir_allwork = 0
                # for weekend
                Dir_ywkend = []
                Dir_monwkend = []
                Dir_allwkend = 0
                # without traffic
                Dir_yntraff = []
                Dir_monntraff = []
                Dir_allntraff = 0
                # weighted
                Dir_yweighted = []
                Dir_monweighted = []
                Dir_allweighted = 0
                # for diffuse radiation
                Rglb_work = 0
                Rglb_wkend = 0
                Rglb_ntraff = 0
                Rglb_weighted = 0
                for mon in range(0,len(sunmapc_arr)): 
                    Pdif = float(cloudy[mon][1])
                    mz_m = []
                    AngIn = []
                    # dir 
                    Dir_work = []
                    Dir_wkend = []
                    Dir_ntraff = []
                    Dir_weighted = []
                    # dir month tempt workday
                    Dir_monworkt = 0
                    Dir_monwkendt = 0
                    Dir_monntrafft = 0
                    Dir_monweightedt = 0
                    for num in range(0,len(sunmapc_arr[mon])):
                        thita = math.radians(90-sunmapc_arr[mon][num][2])
                        alfa = math.radians(sunmapc_arr[mon][num][3])
                        hournow = int(math.floor(sunmapc_arr[mon][num][1]))
                        if hournow < 5 :
                            hournow = 5
                        elif hournow > 20:
                            hournow = 20
                        # two calculate methods by thita copared to 80 degree
                        if thita < 80:
                            # For zenith angles less than 80 degree,
                            # m(θ) = EXP(-0. 000118 * Elev - 1. 638 * 10^-9 * Elev^2) /cos(θ)
                            mz_t = math.exp(-0.000118 * Elev - 1.638 * math.pow(10,-9) * math.pow(Elev,2))/math.cos(thita)
                        else:
                            # get from 
                            index = int(round(thita))-80
                            if index == 10:
                                mz_t = m_thita[9]
                            else:
                                mz_t = m_thita[index]
                        # Angle of incidence (AngInSky θ,α) between the intercepting surface and a given sky sector with a centroid at zenith angle θ and azimuth angle α
                        # AngIn θ,α = acos[Cos(θ)*Cos(Gz)+Sin(θ)*Sin(Gz)*Cos(α-Ga)]
                        angin_t = math.acos(math.cos(thita)*math.cos(Gz)+math.sin(thita)*math.sin(Gz)*abs(math.cos(alfa-Ga)))
                        # calculate Direct rdiation
                        # Dirθ,α = SConst * β^m(θ) * SunDurθ,α * SunGapθ,α * cos(AngInθ,α)
                        gap_count = len(gap_sunmap[mon])
                        if num == gap_count or num > gap_count:
                            gap_tem = gap_sunmap[mon][gap_count-1]
                        else:
                            gap_tem = gap_sunmap[mon][num]
                        Dir_ntrafft = SConst * math.pow(beta,mz_t) * day[mon]*0.5 * gap_tem * math.cos(angin_t)*(1-Pdif)
                        Dir_workt = Dir_ntrafft * float(data[f_id-1,3+hournow])
                        Dir_wkendt = Dir_ntrafft * float(data[f_id-1,19+hournow])
                        Dir_weightedt = Dir_workt * 5/7 + Dir_wkendt * 2/7
                        Dir_monworkt += Dir_workt
                        Dir_monwkendt += Dir_wkendt
                        Dir_monntrafft += Dir_ntrafft
                        Dir_monweightedt += Dir_weightedt
                        mz_m.append(mz_t)
#                         mz_sum += mz_t * Pdif * day[mon] * 0.5
                        # prepare for diffuse radiation for Rglb
                        # Rglb = (SConst Σ(βm(θ))) / (1 - Pdif)
                        Rglb_ntrafft = SConst * math.pow(beta,mz_t) * Pdif * 0.5 / (1-Pdif)
                        Rglb_workt = Rglb_ntrafft * float(data[f_id-1,3+hournow])
                        Rglb_wkendt = Rglb_ntrafft * float(data[f_id-1,19+hournow])
                        Rglb_ntraff += Rglb_ntrafft
                        Rglb_work += Rglb_workt
                        Rglb_wkend += Rglb_wkendt 
                        AngIn.append(angin_t)
                        Dir_ntraff.append(Dir_ntrafft)
                        Dir_work.append(Dir_workt)
                        Dir_wkend.append(Dir_wkendt)
                        Dir_weighted.append(Dir_weightedt)
                    Dir_yntraff.append(Dir_ntraff)
                    Dir_ywork.append(Dir_work)
                    Dir_ywkend.append(Dir_wkend)
                    Dir_yweighted.append(Dir_weighted)
                    Dir_monwork.append(Dir_monworkt)
                    Dir_allntraff += Dir_monntrafft
                    Dir_allwork += Dir_monworkt
                    Dir_allwkend += Dir_monwkendt
                    Dir_allweighted += Dir_monweightedt
                    mz_y.append(mz_m)
                    AngIn_y.append(AngIn)
                Rglb_weighted = Rglb_work * 5/7 + Rglb_wkend *2/7
#                 print('Dir:',Dir_allntraff,Dir_allwork,Dir_allwkend,Dir_allweighted)
#                 print("Rglb:",Rglb_ntraff,Rglb_work,Rglb_wkend,Rglb_weighted)
                skyAngIn = []
                #  Difθ,α = Rglb * Pdif * Dur * SkyGapθ,α * Weightθ,α * cos(AngInθ,α) 
                Dif_secntraff = []
                Dif_secwork = []
                Dif_secwkend = []
                Dif_secweighted = []
                Dif_sumntraff = 0
                Dif_sumwork = 0
                Dif_sumwkend = 0
                Dif_sumweighted = 0
                for nsec in range(0,sky_num):
                    sky_thita = skythitac_arr[nsec]
                    skyangin_t = math.acos(math.cos(sky_thita)*math.cos(Gz)+math.sin(sky_thita)*math.sin(Gz)*abs(math.cos(skyazimuthc_arr[nsec]-Ga)))
                    tempt = gap_sky[nsec] * skyweight_arr[nsec] * math.cos(skyangin_t)
#                     print(Rglb_ntraff,gap_sky[nsec],skyweight_arr[nsec],math.cos(skyangin_t))
#                     time.sleep(1)
                    Dif_ntrafft = Rglb_ntraff * tempt
                    Dif_workt = Rglb_work * tempt
                    Dif_wkendt = Rglb_wkend * tempt
                    Dif_weightedt = Rglb_weighted * tempt
                    skyAngIn.append(skyangin_t)
                    Dif_secntraff.append(Dif_ntrafft)
                    Dif_secwork.append(Dif_workt)
                    Dif_secwkend.append(Dif_wkendt)
                    Dif_secweighted.append(Dif_weightedt)
                    Dif_sumntraff += Dif_ntrafft
                    Dif_sumwork += Dif_workt
                    Dif_sumwkend += Dif_wkendt
                    Dif_sumweighted += Dif_weightedt
#                 print('Dif:',Dif_sumntraff,Dif_sumwork,Dif_sumwkend,Dif_sumweighted)
                writer.writerow([f_id,data[f_id-1,1],data[f_id-1,2],imginfo[1],imginfo[2],Elev,data[f_id-1,6],data[f_id-1,7],
                                 Dir_allntraff,Dir_allwork,Dir_allwkend,Dir_allweighted,
                                 Dif_sumntraff,Dif_sumwork,Dif_sumwkend,Dif_sumweighted,
                                 Dir_allntraff+Dif_sumntraff,Dir_allwork+Dif_sumwork,Dir_allwkend+Dif_sumwkend,Dir_allweighted+Dif_sumweighted])
        outfile.close()
        print('finish')

/datadrive/lzy/Pano/MDJBSV/MDJBSV1_p_pano
/datadrive/lzy/Pano/MDJBSV/MDJBSV2_p_pano
/datadrive/lzy/Pano/MDJBSV/MDJBSV3_p_pano
